In [1]:
# Method to access the transaction data of the routing node: RTL (LND or Eclair) or gRPC
#data_access_method = "gRPC"
data_access_method = "RTL"

if (data_access_method == "gRPC"):

    # Replace with your routing node's certificate, macaroon file path, server port and protobuf file location
    cert_file = r'D:\lnd\node1\tls.cert'
    macaroon_file = r'D:\lnd\node1\data\chain\bitcoin\regtest\admin.macaroon'
    rpcserver = 'localhost:10009'
    protobuf_file_location = r'D:\lnd\proto\lnd\lnrpc' # Location of generated Protobuf files
    
    import sys
    # Ensure that this is the same version of Python that was used to generate the protobuf files
    print ("Ensure that this is the same version of Python that was used to generate the protobuf files")
    print(sys.executable)
    print(sys.version)
    
    sys.path.append(protobuf_file_location)
    
    import grpc
    import os
    from google.protobuf.json_format import MessageToDict
    from lightning_pb2 import ForwardingHistoryRequest
    from lightning_pb2 import ChannelGraphRequest
    from lightning_pb2 import GetInfoRequest
    from lightning_pb2_grpc import LightningStub
    
elif (data_access_method == "RTL"):
    
    # Replace with the name of the Ride The Lightning Forwarding History CSV file
    # Note: the provided 5 days of sample data has been anonymized for privacy. Using actual RTL node data will show the node ids
    RTL_file = "..\Forwarding-history-eclair2.csv"
    #RTL_file = "Forwarding-history-sample.csv"
else: 
    
    print("Unsupported data access method.", data_access_method, " Must be gRPC or RTL")
    sys.exit(1)



In [2]:
import pandas as pd
import networkx as nx
import ipycytoscape


In [3]:
def date_to_unix_timestamp(year, month, day, hour=0, minute=0, second=0):
    """Convert a date to a Unix timestamp."""
    dt = datetime.datetime(year, month, day, hour, minute, second)
    return int(dt.timestamp())

In [4]:
if (data_access_method == "gRPC"):
    
    import datetime

    # Set the required date/time range for gRPC. Examples:
    start = date_to_unix_timestamp(2023, 1, 1)  # January 1, 2023, at 00:00:00
    end = date_to_unix_timestamp(2023, 12, 31, 23, 59, 59)  # January 31, 2023, at 23:59:59

In [5]:
# Function to create a gRPC channel with the LND routing node
def create_grpc_channel(cert_file, macaroon_file):
    try:
        with open(cert_file, 'rb') as f:
            cert = f.read()
        with open(macaroon_file, 'rb') as f:
            macaroon = f.read().hex()
    
        credentials = grpc.ssl_channel_credentials(cert)
        channel = grpc.secure_channel('localhost:10009', credentials)
        return channel, macaroon
    except (FileNotFoundError, IOError) as e:
        print(f"Error reading cert or macaroon file: {e}")
        sys.exit(1)
    except Exception as e:
        print(f"Unexpected error in creating gRPC channel: {e}")
        sys.exit(1)

In [6]:
def get_node_info(stub, macaroon):
    try:
        request = GetInfoRequest()
        metadata = [('macaroon', macaroon)]
        response = stub.GetInfo(request, metadata=metadata)
        return response
    except grpc.RpcError as e:
        print(f"gRPC error: {e.details()}")
        sys.exit(1)
    except Exception as e:
        print(f"Unexpected error in fetching node info: {e}")
        sys.exit(1)

In [7]:
# Function to get forwarding history
def get_forwarding_history(stub, start_time, end_time, macaroon):
    try:
        # Create a request with the specified time range (in Unix timestamp)
        request = ForwardingHistoryRequest(start_time=start_time, end_time=end_time, num_max_events=10000)
        #request = ForwardingHistoryRequest()
    
        # Include the macaroon in the metadata
        metadata = [('macaroon', macaroon)]

        # Make the gRPC call
        response = stub.ForwardingHistory(request, metadata=metadata)
    
        # Convert response to dictionary
        events = [MessageToDict(event) for event in response.forwarding_events]
    
        # Return as pandas DataFrame
        return pd.DataFrame(events)
    except grpc.RpcError as e:
        print(f"gRPC error: {e.details()}")
        sys.exit(1)
    except Exception as e:
        print(f"Unexpected error getting forwarding history info: {e}")
        sys.exit(1)
        


In [8]:
def get_network_graph(stub, macaroon):
    
    try:
        # Include the macaroon in the metadata
        metadata = [('macaroon', macaroon)]
    
        request = ChannelGraphRequest(include_unannounced=True)
        response = stub.DescribeGraph(request, metadata=metadata)
        return response
    except grpc.RpcError as e:
        print(f"gRPC error: {e.details()}")
        sys.exit(1)
    except Exception as e:
        print(f"Unexpected error while getting the network graph: {e}")
        sys.exit(1)
        

In [9]:
def create_node_id_to_alias_map(graph):
    node_id_to_alias = {}
    for node in graph.nodes:
        node_id_to_alias[node.pub_key] = node.alias
    return node_id_to_alias

In [10]:
def create_channel_id_to_node_ids_map(graph):
    channel_id_to_node_ids = {}
    for edge in graph.edges:
        channel_id_to_node_ids[str(edge.channel_id)] = (edge.node1_pub, edge.node2_pub)
    return channel_id_to_node_ids

In [11]:
def get_alias_for_channel(channel_id, channel_id_to_node_ids, node_id_to_alias, exclude_node_pub_key):
    node_ids = channel_id_to_node_ids.get(str(channel_id), None)
    if node_ids:
        # Filter out the excluded node alias and return the other alias
        aliases = [node_id_to_alias.get(node_id) for node_id in node_ids if node_id != exclude_node_pub_key]
        if aliases:
            return aliases[0]
    return None


In [12]:
def load_data(filepath):
    try:
        df = pd.read_csv(filepath, encoding='utf-8')

        # Define the column name mappings for LND and Eclair
        lnd_columns = {
            'amt_in_msat': 'amt_in_msat',
            'amt_out_msat': 'amt_out_msat',
            'fee_msat': 'fee_msat',
            'alias_in': 'alias_in',
            'alias_out': 'alias_out'
        }
        eclair_columns = {
            'amountIn': 'amt_in_msat',
            'amountOut': 'amt_out_msat',
            'fromChannelAlias': 'alias_in',
            'toChannelAlias': 'alias_out'
        }

        # Detect the format (LND or Eclair) based on the column names
        #print (df.columns)
        if set(lnd_columns.keys()).issubset(df.columns):
            column_map = lnd_columns
        elif set(eclair_columns.keys()).issubset(df.columns):
            column_map = eclair_columns
            # Calculate fee_msat as the difference between amountIn and amountOut
            df['fee_msat'] = df['amountIn'] - df['amountOut']
        else:
            raise ValueError("CSV format not recognized. Please ensure it's either LND or Eclair format.")

        # Rename the columns
        df.rename(columns=column_map, inplace=True)

        return df

    except FileNotFoundError:
        print(f"Error: The file {filepath} was not found.")
    except pd.errors.EmptyDataError:
        print(f"Error: The file {filepath} is empty.")
    except pd.errors.ParserError:
        print(f"Error: The file {filepath} does not appear to be a valid CSV or is improperly formatted.")
    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

    return None

In [13]:
def add_nodes_and_edges(G, df):
    """Add nodes and edges to the graph G based on the dataframe df."""
    for index, row in df.iterrows():
        
        # Convert node names to strings to handle numerical aliases
        u = str(row['alias_in'])
        v = str(row['alias_out'])
        
        # Convert 'amt_out_msat' and 'fee_msat' to integers
        weight = int(row['amt_out_msat'])
        fee = int(row['fee_msat'])
                
        # Check if there's already an edge (i.e., a previous transaction between these channels)
        if G.has_edge(u, v):
            edge_data = G[u][v]
            edge_data['weight'] += weight
            edge_data['fees'].append(fee)
        else:
            G.add_edge(u, v, weight=weight, fees=[fee])
    
    # Average out the fees and compute total transactions
    total_transactions = sum(nx.get_edge_attributes(G, 'weight').values())
    for u, v, d in G.edges(data=True):
        d['avg_fee'] = sum(d['fees']) / len(d['fees'])
        d['normalized_fee'] = d['avg_fee'] / total_transactions * 10000

    return total_transactions


In [14]:
def assign_classes_to_nodes(cyto_graph, df):
    # Ensure numeric columns are of the correct type
    df['fee_msat'] = pd.to_numeric(df['fee_msat'], errors='coerce').fillna(0)
    df['amt_out_msat'] = pd.to_numeric(df['amt_out_msat'], errors='coerce').fillna(0)

    # Calculate total fees and total transactions for each node
    total_fees = df.groupby('alias_out')['fee_msat'].sum()
    total_transactions = df.groupby('alias_out')['amt_out_msat'].sum()

    # Normalize the values
    max_fee = total_fees.max()
    max_transaction = total_transactions.max()
    normalized_fees = total_fees / max_fee * 10000
    normalized_transactions = total_transactions / max_transaction * 10000

    # Adjust thresholds based on the distribution of your data
    fee_thresholds = [100,5000]  # example thresholds for low, medium, high fee
    transaction_thresholds = [500, 1000, 3000]  # example thresholds for tiny, small, medium, large transactions

    # Iterate over nodes to assign classes
    for node in cyto_graph.graph.nodes:
        node_alias = node.data['id']
        classes = []

        # Assign fee classes based on normalized fees
        fee = normalized_fees.get(node_alias, 0)
        #print (node_alias, fee)
        if fee == 0:
            classes.append('no_fee')
        elif fee <= fee_thresholds[0]:
            classes.append('low_fee')
        elif fee <= fee_thresholds[1]:
            classes.append('medium_fee')
        else:
            classes.append('high_fee')

        # Assign transaction size classes based on normalized transactions
        trans = normalized_transactions.get(node_alias, 0)
        #print (node_alias, trans)
        if trans <= transaction_thresholds[0]:
            classes.append('tiny_transaction')
        elif trans <= transaction_thresholds[1]:
            classes.append('small_transaction')
        elif trans <= transaction_thresholds[2]:
            classes.append('medium_transaction')
        else:
            classes.append('large_transaction')

        node.classes = ' '.join(classes)

In [15]:
def assign_classes_to_edges(cyto_graph, G, total_transactions):
    # Normalize transactions for edges
    for u, v, d in G.edges(data=True):
        d['weight'] = int(d.get('weight', 0))  # Convert to integer
        d['normalized_transaction'] = d['weight'] / total_transactions * 10000

    # Iterate over edges to assign classes
    for edge in cyto_graph.graph.edges:
        u, v = edge.data['source'], edge.data['target']
        d = G[u][v]
        
        normalized_transaction = d['normalized_transaction']

        if normalized_transaction < 1:
            edge.classes = 'tiny_volume'
        elif normalized_transaction < 10:
            edge.classes = 'small_volume'
        elif normalized_transaction < 100:
            edge.classes = 'medium_volume'
        else:
            edge.classes = 'large_volume'

In [16]:
# Set styles based on classes
def set_styles (cyto_graph):
    
    cyto_graph.set_style([
        {
            'selector': 'node.no_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'red'
            }
        },
        {
            'selector': 'node.low_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'orange'
            }
        },
        {
            'selector': 'node.medium_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'yellow'
            }
        },
        {
            'selector': 'node.high_fee',
            'style': {
                'content': 'data(id)',
                'background-color': 'green'
            }
        },
            {
            'selector': 'node.tiny_transaction',
            'style': {
                'width': '10px',
                'height': '10px'
            }
        },
        {
            'selector': 'node.small_transaction',
            'style': {
                'width': '20px',
                'height': '20px'
            }
        },
        {
            'selector': 'node.medium_transaction',
            'style': {
                'width': '30px',
                'height': '30px'
            }
        },
        {
            'selector': 'node.large_transaction',
            'style': {
                'width': '50px',
                'height': '50px'
            }
        },
        {
            'selector': 'edge.tiny_volume',
            'style': {
                'width': '0px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.small_volume',
            'style': {
                'width': '1px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.medium_volume',
            'style': {
                'width': '2px',
                'line-color': 'grey'
            }
        },
        {
            'selector': 'edge.large_volume',
            'style': {
                'width': '3px',
                'line-color': 'black'
            }
        }
    ])


In [17]:
if (data_access_method == "RTL"):
    df = load_data (RTL_file)
      
else:

    channel, macaroon = create_grpc_channel(cert_file, macaroon_file)
    stub = LightningStub(channel)

    # Extract the public key of the routing node
    node_info = get_node_info(stub, macaroon)
    routing_node_pub_key = node_info.identity_pubkey

    # Fetch forwarding history
    df = get_forwarding_history(stub, start, end, macaroon)

    # Fetch the graph and create the maps
    graph = get_network_graph(stub, macaroon)

    node_id_to_alias = create_node_id_to_alias_map(graph)
    channel_id_to_node_ids = create_channel_id_to_node_ids_map(graph)

    # Exclude the routing node itself from the network graph
    exclude_node_pub_key = routing_node_pub_key

    # When applying the aliases to the DataFrame
    df['alias_in'] = df['chanIdIn'].apply(lambda x: get_alias_for_channel(x, channel_id_to_node_ids, node_id_to_alias, exclude_node_pub_key))
    df['alias_out'] = df['chanIdOut'].apply(lambda x: get_alias_for_channel(x, channel_id_to_node_ids, node_id_to_alias, exclude_node_pub_key))

    # ALign the column names returned from gRPC with the CSV column names
    df = df.rename(columns={
    'feeMsat': 'fee_msat',
    'amtInMsat': 'amt_in_msat',
    'amtOutMsat': 'amt_out_msat',
    'timestampNs': 'timestamp_ns'
    })

# Check if the data was loaded successfully
if df is None:
    # If df is None, print an error message and exit
    print("Transaction data could not be loaded. Please check the file path and format or API connection.")
    sys.exit(1)

# Create a new Cytograph
G = nx.Graph()

total_transactions = add_nodes_and_edges(G, df)

cyto_graph = ipycytoscape.CytoscapeWidget()
cyto_graph.graph.add_graph_from_networkx(G)

cyto_graph.set_layout(name='circle')

assign_classes_to_nodes (cyto_graph, df)
assign_classes_to_edges (cyto_graph, G, total_transactions)

set_styles (cyto_graph)

# Note: 
# Ensure that ipytoscape is enabled in Jupyter Lab or Jupyter Notebook, in a Conda shell run the following
# jupyter nbextension enable --py --sys-prefix ipycytoscape
# Start Jupyter with "jupyter notebook (or jupyter lab) --NotebookApp.iopub_data_rate_limit=1.0e10", in Powershell if datarate is exceeded
# This can happen with large numbers of transactions

cyto_graph

CytoscapeWidget(cytoscape_layout={'name': 'circle'}, cytoscape_style=[{'selector': 'node.no_fee', 'style': {'c…